In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2

# Transformer 深入淺出

Google Research 和 Google Brain 的成員最初在 2017 年的論文《[Attention is all you need](https://arxiv.org/abs/1706.03762?context=cs)》中提出了 Transformer。目前是 NLP 中最受歡迎的架構之一。

Transformer 由 Encoder 和 Decoder 两部分组成，Encoder 接受 **输入** 然後生成 **向量** 表示這個输入，Decoder 接受 **向量** 然後逐一生成 **输出**。

![transformer in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoder-decoder-1-1024x275.png)

Transformer 超越了基於 LSTM 和 RNN 的其他現有架構，獲得了更高評價和更快的訓練速度。Transformer 的注意力機制是一種「word-to-word」的操作，它會找出序列中每個詞與其他詞之間的關係。

![Original Transformer Architecture, Source: Attention is all you need](https://www.alexisalulema.com/wp-content/uploads/2022/08/image-695x1024.png)

在本文中，將以最簡單的方式解釋 Transformer 的架構和內部運作。

## 分詞器 (Tokenizer)

首先，分詞器負責為模型輸入準備。大多數分詞器有兩種版本：完整的 Python 實現和基於 Rust 庫的“快速”實現。“快速”實現具有以下優點：

1. 特別是在批量分詞時顯著加速；

2. 提供額外的方法在原始字符串和標記空間之間進行映射（例如，獲取給定字符的標記索引或對應於給定標記的字符範圍）。

後面的課程會利用到這兩項優點，暫時無須深入了解。

分詞器 (Tokenizer) 通過將文本正規化來將輸入轉換為標記 (Token)；此外，它還會提供一個整數表示（基於現有詞彙庫），該表示將用於 Embedding 過程。

我們先加載一個分詞器，然後觀察其包含的資訊：

In [2]:
# 載入 BERT tokenizer
from transformers import AutoTokenizer
from pprint import pprint

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Users/jonas/.pyenv/versions/learning-transformer-with-huggingface-3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# vocab_szie: 30522
# model_max_length: 512
tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

Tokenizater 包含以下重要資訊

* `vocab_size`：詞彙表的大小

* `model_max_length`：輸入到模型的最大標記數

* 特殊的 token，如

  * 0: [PAD]
  * 100: [UNK]
  * 101: [CLS]
  * 102: [SEP]
  * 103: [MASK]


### 詞彙庫 (Vocabulary)

顯示詞彙庫部分內容，從第 1 個詞開始到第 10 個詞，而且有些字僅有子詞 (subword)。

In [4]:
# 顯示挑選的前 10 個標記 (token)
pprint(list(tokenizer.vocab.keys())[0:10], compact=True)

['piss', 'mom', '##石', 'instrumentation', 'cannabis', 'images', '[unused976]',
 '##rya', 'mohan', 'informal']


### 標記 (Token)

標記是分詞器的輸出，它是一個整數，代表字典中的一個詞。標記是 Embedding 層的輸入。

In [5]:
raw_inputs = [
    "I've been waiting for a this course my whole life.",
    "I hate this so much!",
]
# 將原始文字轉換成標記
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
)

pprint(inputs, compact=True)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[ 101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 2023, 2607, 2026, 2878,
         2166, 1012,  102],
        [ 101, 1045, 5223, 2023, 2061, 2172,  999,  102,    0,    0,    0,    0,
            0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


1. 因第二句子長度不夠，所以會被補上 [PAD] token
2. Attention Mask 會註記哪些是 padding 的 token

我們也可以將標記轉換回原始文本，這樣我們就可以看到模型的輸出是什麼樣子的。

In [6]:
tokenizer.decode(inputs["input_ids"][0])

"[CLS] i ' ve been waiting for a this course my whole life. [SEP]"

In [7]:
tokenizer.decode(inputs["input_ids"][1])

'[CLS] i hate this so much! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

其中 [CLS] 和 [SEP] 是特殊 token，[CLS] 用於句子分類任務，[SEP] 用於分隔兩個句子。

### 多語分詞器 (Multilingual Tokenizer)

如果將此模型應用在中文文本上，會發生什麼狀況呢？

In [8]:
raw_inputs = [
    "我門正在學習目前正夯的變形金剛模型！",
    "你們喜歡這個課程嗎？"
]
# 將原始文字轉換成標記
inputs = tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
)

由於 Tokenizer 是基於英文訓練的，所以它無法正確處理中文文本。這是因為中文文本的分詞方式與英文不同。因此，我們需要使用中文分詞器來處理中文文本。

我們嘗試用多語分詞器來處理中文文本，看看會發生什麼事情。

In [9]:
# 載入 Multilingual BERT tokenizer
mbert_model_name = "google-bert/bert-base-multilingual-uncased"
multilingual_tokenizer = AutoTokenizer.from_pretrained(mbert_model_name)

In [10]:
# vocab_szie: 30522 > 105879
# model_max_length: 512
multilingual_tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-multilingual-uncased', vocab_size=105879, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

模型的最大長度不變，但是詞彙庫的大小增加了。這是因為多語分詞器包含了中文詞彙。

In [11]:
# 將原始文字轉換成標記
multilingual_inputs = multilingual_tokenizer(
  raw_inputs,
  padding=True,         # Activates and controls padding
  truncation=True,      # Activates and controls truncation
  return_tensors="pt",  # Return PyTorch tensor objects
)

In [12]:
# 將標記轉換回文字
multilingual_tokenizer.decode(multilingual_inputs["input_ids"][0])

'[CLS] 我 門 正 在 學 習 目 前 正 夯 的 變 形 金 剛 模 型 ！ [SEP]'

In [13]:
# 將標記轉換回文字
multilingual_tokenizer.decode(multilingual_inputs["input_ids"][1])

'[CLS] 你 們 喜 歡 這 個 課 程 嗎 ？ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

## Input Embedding

Transformer 架構首先是 Input Embedding 層，它將輸入序列轉換為維度 $\ d_{model} = 512 $ 的向量。

![Input Embedding](https://www.alexisalulema.com/wp-content/uploads/2022/08/input.embedding.png)

> $\ d_{model} = 512 $ 的值是由架構設計者設定的，這個維度的值可以根據目標進行修改。

以我們使用的模型 `bert-base-uncased` 為例，它的 $\ d_{model} = 768 $。對於每個唯一的標記 ID（即 BERT 分詞器詞彙表中的 30,522 個單詞和子詞），BERT 模型包含一個嵌入 (Embedding)，該嵌入被訓練來表示那個特定的標記。模型中的嵌入層負責將標記映射到它們對應的嵌入。

![](https://cdn.prod.website-files.com/6064b31ff49a2d31e0493af1/66d06d2f219c5eab928c6b5b_AD_4nXdL2BUY6asFzNdhx_FYCFp6DNBRCwLx_XCALqjkUueNttpIa0WPQWRzUxSNvDBdyU6U3r5unc1OJu4-4DxbNAXeaXlS7Y9BW2-igGX91VVZHRSlwLYYw0dE0m5DPrw29A3RNnVp5hV9S3ljW7GcLYRMPaX0.png)

In [14]:
from transformers import AutoModel

# 載入 BERT model
model = AutoModel.from_pretrained(
  model_name,
)

# 取得 BERT embeddings
model.embeddings

BertEmbeddings(
  (word_embeddings): Embedding(30522, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

模型 `word_embeddings.weight` 屬性可以用來查看和單一對應嵌入層，該層的屬性包含標記嵌入（即 BERT 分詞器詞彙表中每個標記的嵌入）。

In [15]:
model.embeddings.word_embeddings.weight

Parameter containing:
tensor([[-0.0102, -0.0615, -0.0265,  ..., -0.0199, -0.0372, -0.0098],
        [-0.0117, -0.0600, -0.0323,  ..., -0.0168, -0.0401, -0.0107],
        [-0.0198, -0.0627, -0.0326,  ..., -0.0165, -0.0420, -0.0032],
        ...,
        [-0.0218, -0.0556, -0.0135,  ..., -0.0043, -0.0151, -0.0249],
        [-0.0462, -0.0565, -0.0019,  ...,  0.0157, -0.0139, -0.0095],
        [ 0.0015, -0.0821, -0.0160,  ..., -0.0081, -0.0475,  0.0753]],
       requires_grad=True)

雖然從表面上看不明顯，但每個標記 ID 的嵌入並不只是隨機數字。這些值是在訓練 BERT 模型時學習到的，這意味著每個嵌入都編碼了模型對該特定標記的理解。

## 位置編碼向量 (Positional Encoding)

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/positional.encoding.png)

除了前面描述的標記嵌入外，Transformer 還依賴於位置編碼 (Positional Encoding)。標記嵌入用於代表模型的每個可能的單詞或子詞，而位置嵌入則表示輸入序列中每個標記的位置。

![](https://cdn.prod.website-files.com/6064b31ff49a2d31e0493af1/66d06d30496ac93f233f13a6_AD_4nXeVGAn7H8YRdRk0S9Z6PqBiiVaJ2Aykuca5ZA0rWsj7-IX9AJncsiGBZ2thKwnAxO64_fKmSrDv9dWiuonp-wSMEnVre34FMt3QjWdpwchjUXNfv0Ry23qYuvzYjmESaWiwHCUFsmqsQ0SyfLi6ub0syWEg.png)

初始序列中的每個詞都必須具有位置編碼（PE）信息，但由於原論文的主要重點是注意力機制，因此這個向量的生成，作者找到了一種巧妙的方法，使用正弦和餘弦值表示位置編碼。

$\ PE_{\text{sin}}(\text{pos}, 2i) = \sin\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $

$\ PE_{\text{cos}}(\text{pos}, 2i+1) = \cos\left(\frac{\text{pos}}{10000^{\frac{2i}{d_{\text{model}}}}}\right) $

以下是我們使用上述公式，計算第 3 個位置的位置編碼：

In [16]:
import math

d_model = 768

def positional_encoding(position):
    pe = [None] * d_model

    for i in range(0, d_model, 2):
        pe[i] = math.sin(position / (10000 ** ((2 * i) / d_model)))
        pe[i + 1] = math.cos(position / (10000 ** ((2 * i) / d_model)))

    return pe

# 計算第 3 個位置的 positional encoding
pprint(positional_encoding(3), compact=True)

[0.1411200080598672, -0.9899924966004454, 0.27837998117302903,
 -0.9604710230309418, 0.40414136603532824, -0.9146965377976998,
 0.517305716423722, -0.8558006752482378, 0.6173581934738708,
 -0.7866821854794215, 0.704246484600274, -0.7099555541920608,
 0.7782725224195125, -0.6279266524418035, 0.8399987525023596,
 -0.5425883299468205, 0.8901692262613478, -0.4556300567535832,
 0.9296448405538298, -0.3684568773026832, 0.9593514800610861,
 -0.282213992751252, 0.9802395338823054, -0.19781419618975907,
 0.993253167134793, -0.11596614150993839, 0.999307764654284,
 -0.03720203625688118, 0.9992740773014782, 0.03809617347292795,
 0.9939677561484435, 0.10967269367180021, 0.9841431312738992,
 0.17737614599039198, 0.9704902638019913, 0.24114030742607345,
 0.9536344621455892, 0.3009672949147346, 0.9341376005608856, 0.3569130751574553,
 0.9125007075335518, 0.40907512604748686, 0.8891674026610501,
 0.45758204733687047, 0.8645278542932224, 0.5025849074048651,
 0.8389230076891356, 0.5442501145335796, 0.81

實際上，BERT 的位置編碼是透過訓練學習到的，而不是使用上述公式計算的。

In [17]:
pprint(model.embeddings.position_embeddings.weight[3], compact=True)

tensor([-4.1949e-03, -1.1852e-02, -2.1180e-02, -9.3056e-03, -1.0164e-02,
         1.7667e-02, -5.2579e-03, -5.9056e-03, -5.2515e-03, -1.9121e-02,
         2.3867e-02, -2.7827e-02,  1.8384e-02, -1.1762e-02, -2.0740e-03,
         7.1641e-03, -2.9109e-03,  1.9358e-02,  8.1694e-04, -5.6441e-03,
        -7.8681e-03, -2.5424e-02, -8.3934e-03,  1.3958e-02,  2.4538e-03,
        -6.5318e-03, -2.3111e-02,  1.0861e-02, -1.0973e-02,  1.7499e-02,
         1.5851e-02, -1.4915e-02,  2.6499e-03,  3.0319e-03, -8.4172e-03,
        -1.6653e-02, -3.2846e-03, -8.7255e-03, -4.6161e-03, -2.7917e-03,
         1.5829e-02,  2.3712e-03, -1.5293e-02, -5.9103e-03, -9.5935e-04,
        -1.5983e-02,  9.4848e-04,  1.0567e-02,  2.0371e-02, -2.6296e-03,
        -3.3435e-03,  9.3798e-03,  9.9587e-03, -5.1266e-03, -3.5115e-03,
         1.9099e-02,  1.5399e-02,  7.0687e-03,  1.6475e-02, -1.5982e-02,
        -6.7953e-03, -1.3570e-03, -1.7052e-02, -8.2985e-03, -6.3168e-03,
        -9.0573e-03, -8.9459e-04, -6.1750e-03,  6.3

最後，Input Embedding 產生的編碼向量應該加上位置編碼向量，然後將結果輸入到編碼器/解碼器區塊中：

$\ Embedding_{output} = Embedding_{initial} + PE_{word-position} $

以上是簡化的說明，實際上 BERT 模型還額外加上 Token Type Embedding，這不在我們的討論範圍內。

![](https://cdn.prod.website-files.com/6064b31ff49a2d31e0493af1/66d06d2f71f03e028aa4698f_AD_4nXeBMY4_9hogpj-S8elt4Lm0PYtwN5KWb3drhXV0mUVvG5FwCNdkfTQcvAQGwBgeYfkEaZNUKN6YKPaFrNayvn4hcDUx-FXsW9bFyPNDM86Rjt5E0kyIkKhJowh--90eICovok_fBGdLb32mSUL7pljzM518.png)